In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
import pprint
import glob
from pathlib import Path
import pickle
import random

import torch
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import trimesh

from utils import general_utils
from utils import visualization_tools
from utils.eval_utils import eval_metrics
#from evaluation import compute_iou_2d, compute_iou_2d_given_pose, compute_iou_3d, compute_chamfer_L1

In [4]:
# displays meshes at the predicted pose
def show_meshes(input_dir_img, input_dir_mesh, refined_models_dir, gt_shapes_dict, device, only_show_instances=None, num_novel_view=3,
                metrics_to_eval=["2d_iou_input", "2d_iou_multi", "3d_iou","chamfer_L1"], num_sample_points=900000):
    

    # combining all cached predicted poses
    pred_poses_dict = {}
    pred_pose_paths = list(Path(refined_models_dir).rglob('pred_poses.p'))
    for pred_pose_path in pred_pose_paths:
        curr_cache = pickle.load(open(pred_pose_path, "rb"))
        pred_poses_dict = {**pred_poses_dict, **curr_cache}
    
    # getting paths of all processed meshes, keyed by instance name
    processed_paths = list(Path(refined_models_dir).rglob('*.obj'))
    processed_paths_dict = {}
    for path in processed_paths:
        instance_name = str(path).split('/')[-1][:-4]
        if instance_name not in processed_paths_dict:
            processed_paths_dict[instance_name] = [path]
        else:
            processed_paths_dict[instance_name].append(path)
    
    for instance_name in processed_paths_dict:
        if only_show_instances is None or instance_name in only_show_instances:
            for processed_mesh_path in processed_paths_dict[instance_name]:
                print(processed_mesh_path)
            
                input_image = Image.open(os.path.join(input_dir_img, instance_name+".png"))
                original_mesh_path = os.path.join(input_dir_mesh, instance_name+".obj")
                with torch.no_grad():
                    mesh_original = general_utils.load_untextured_mesh(original_mesh_path, device)
                    mesh_processed = general_utils.load_untextured_mesh(processed_mesh_path, device)
                
                pred_dist = pred_poses_dict[instance_name]['dist']
                pred_elev = pred_poses_dict[instance_name]['elev']
                pred_azim = pred_poses_dict[instance_name]['azim']
                
                # computing performance metrics
                if len(metrics_to_eval) > 0:
                    mesh_original_trimesh = trimesh.load(original_mesh_path)
                    mesh_processed_trimesh = trimesh.load(processed_mesh_path)
                    
                    gt_mesh_path = gt_shapes_dict[instance_name]
                    with torch.no_grad():
                        mesh_gt = general_utils.load_untextured_mesh(gt_mesh_path, device)
                    mesh_gt_trimesh = trimesh.load(gt_mesh_path)
                    
                    metrics_dict_before, _ = eval_metrics(input_image, mesh_original_trimesh, mesh_original, mesh_gt_trimesh, mesh_gt,
                                                          device, metrics_to_eval, pred_azim, pred_elev, pred_dist, num_sample_points=num_sample_points)
                    metrics_dict_after, _ = eval_metrics(input_image, mesh_processed_trimesh, mesh_processed, mesh_gt_trimesh, mesh_gt,
                                                         device, metrics_to_eval, pred_azim, pred_elev, pred_dist, num_sample_points=num_sample_points)
                    
                    print("Metrics before refinement -- {}".format(metrics_dict_before))
                    print("Metrics after refinement -- {}".format(metrics_dict_after))
                
                visualization_tools.show_refinement_results(input_image, mesh_original, mesh_processed, pred_dist, pred_elev, pred_azim, device, num_novel_view)

In [8]:
# shapenet
class_name = "02933112"
gt_shapes_list_path = "data/gt_shapes_path_lists/shapenet_{}_gt_shapes.lst".format(class_name)
input_dir_img = "data/input_images/pytorch3d_shapenet_renders/{}/rgba".format(class_name)
refinement_type = "gt_pose"
#refinement_type = "ablation_2"

# atlasnet
#input_dir_mesh = "/home/svcl-oowl/brandon/research/AtlasNet/data/pytorch_3d_render_recs/{}".format(class_name)
#refined_models_dir = "data/refinements/shapenet_atlasnet_refinements/gt_pose/{}".format(class_name)
# pix2mesh
#input_dir_mesh = "/home/svcl-oowl/brandon/research/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/{}/rgba".format(class_name)
#refined_models_dir = "data/refinements/shapenet_pix2mesh_refinements/gt_pose/{}".format(class_name)
# occnet
input_dir_mesh = "/home/svcl-oowl/brandon/research/occupancy_networks/out/pytorch3d_renders/{}/generation/meshes".format(class_name)
refined_models_dir = "data/refinements/shapenet_occnet_refinements/{}/{}".format(refinement_type, class_name)

# pix3d occnet gt
#class_name = "wardrobe"
#input_dir_mesh = "/home/svcl-oowl/brandon/research/occupancy_networks/out/pix3d/{}/generation/meshes".format(class_name)
#input_dir_img = "data/input_images/pix3d_images_processed_filtered/{}/rgba".format(class_name)
#gt_shapes_list_path = "data/gt_shapes_path_lists/pix3d_{}_gt_shapes.lst".format(class_name)
#refined_models_dir = "data/refinements/pix3d_occnet_refinements/gt_pose/{}".format(class_name)

In [9]:
device = torch.device("cuda:0")
gt_shapes_dict = {}
with open(gt_shapes_list_path, 'r') as f:
    f = f.read().split('\n')
    for line in f:
        if line != "":
            gt_shapes_dict[line.split(" ")[0]] = line.split(" ")[1]

In [10]:
#instances = ['fb402a36e91ab1b44e7761521d3c6953', 'ddd6c2a51c25036c8a43cd65b179a1ff', 'd56fba80d39bdff738decdbba236bc1d', 'f8d8b590a0dcd399718ac2a6ddb54499', 'f3cbfb52ea1c907a850e00840470903a', 'f25ffb9cf92236fb9671f5163e7f6535', 'e332fb3eb2c4016ec1f9d235878ff0a9', 'd54a694d514b1911844ac48bcfce34', 'd4cc520929730248642fa8402286a797', 'fbc7afa847c30a4c183bb3a05fac486f', 'dbd589812bda0b1ebab624e35355496d', 'f6b96f56212f55363023a5c0cae3fffe', 'e4bdcd6baad501ab2a8b9d468136b0a1', 'ef6db5bcb6bb96ddd2f0cc036969ee4f', 'f2f6684e930651df3dffb45955223d25', 'e9704a563a5a0a3f5a4b8d382b9decda', 'e53547a01129eef87eda1e12bd28fb7', 'e06d3e6c1fb4b208cb7c15fd62c3982e', 'e4b0599a9d06f7ae39cb1b92881e8e76', 'fb0f2907b977e7cb67c5e3041553656b']
#instances = ['1359', '1422', '0649', '0581', '0863', '0859', '2868', '2475', '0770', '0652', '0647', '0966', '0661', '2931', '0927', '3313', '2969', '0744', '2944', '0672']
#instances = None

In [12]:
#instances = random.sample([str(path).split('/')[-1].replace('.obj',"") for path in list(Path(refined_models_dir).rglob('*.obj'))], 180)
instances=["79a8b5fdd40e1b1d20768660cf080d12", "eb2843ff62280f5320768660cf080d12", "5ff47fe4724d0c9320768660cf080d12 "]
#m = ["2d_iou_input", "2d_iou_multi", "3d_iou","chamfer_L1"]
m = []
show_meshes(input_dir_img, input_dir_mesh, refined_models_dir, gt_shapes_dict, device, only_show_instances=instances, metrics_to_eval=m, num_sample_points=500000)

In [ ]:
# show training loss info
# getting paths of all loss info pickled files
input_dir_mesh = cfg['dataset']['input_dir_mesh']
output_dir = "{}_{}".format(input_dir_mesh, exp_name)
cached_loss_info_paths = list(Path(output_dir).rglob('loss_info.p'))

for path in cached_loss_info_paths:
    dataset_loss_info = pickle.load(open(path, "rb"))
    for instance_name in dataset_loss_info:
        loss_info = dataset_loss_info[instance_name]
        loss_info.plot.line(x='iteration', y='total_loss', title="{}".format(instance_name))
        #loss_info.plot.line(x='iteration', y='sil_loss')
        #loss_info.plot.line(x='iteration', y='img_sym_loss')
        #loss_info.plot.line(x='iteration', y='vertex_sym_loss')
        #loss_info.plot.line(x='iteration', y='l2_loss')
        #loss_info.plot.line(x='iteration', y='lap_smoothness_loss')
        #loss_info.plot.line(x='iteration', y='normal_consistency_loss')
        #loss_info.plot.line(x='iteration', y='semantic_dis_loss')